In [1]:
# default_exp contacts

# contacts

> API details.

In [2]:
# hide
from nbdev.showdoc import *

In [3]:
# hide
# export

import time
import os
import json
import requests
from dotenv import load_dotenv
from am4894hubspot.search import hubspot_search
load_dotenv()

True

In [4]:
hapikey = os.environ.get('HUBSPOT_API_KEY','demo')
#hapikey = 'demo'

In [5]:
# export


def contact_exists(value, property='email', operator='EQ', hapikey=None) -> bool:
    """
    Check if a contact exists.
    """
    r = hubspot_search(value, property=property, operator=operator, hapikey=hapikey)
    if r.get('total',0) > 0:
        return True
    else:
        return False



In [6]:
# export


def get_contact(value, property='email', operator='EQ', hapikey=None, properties=None):
    """
    Get contact.
    """
    if hapikey is None:
        hapikey = os.environ.get('HUBSPOT_API_KEY','demo')
    r = hubspot_search(value, property=property, operator=operator, hapikey=hapikey, properties=properties)
    if 'total' in r:
        if r['total'] > 0:
            contact = r['results'][0]
            return contact
        else:
            return None
    else:
        return None

    

In [11]:
# export


def get_contact_by_id(id, hapikey=None, properties=None):
    """
    Get contact by id.
    """
    if hapikey is None:
        hapikey = os.environ.get('HUBSPOT_API_KEY','demo')
    endpoint = 'https://api.hubapi.com/crm/v3/objects/contacts/'
    url = f'{endpoint}/{id}'
    querystring = {"hapikey": hapikey, "properties": properties}
    headers = {"accept": "application/json"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    
    return response



In [7]:
# export


def create_contact(contact=None, hapikey=None):
    """
    Create a contact.
    """
    if contact is None:
        contact = dict(email='sometestuser@example.com')
    if hapikey is None:
        hapikey = os.environ.get('HUBSPOT_API_KEY','demo')
    endpoint = 'https://api.hubapi.com/contacts/v1/contact/'
    url = f'{endpoint}?hapikey={hapikey}'
    headers = {
        "Content-Type": "application/json"
    }
    data = json.dumps({"properties": [{"property": k,"value": contact[k]} for k in contact]})
    r = requests.post(url=url, data=data, headers=headers)
    return r



In [8]:
# export 


def delete_contact(vid=None, hapikey=None):
    """
    Delete a contact.
    """
    if hapikey is None:
        hapikey = os.environ.get('HUBSPOT_API_KEY','demo')
    endpoint = f'https://api.hubapi.com/contacts/v1/contact/vid/{vid}'
    url = f'{endpoint}?hapikey={hapikey}'
    headers = {
        "Content-Type": "application/json"
    }
    r = requests.delete(url=url, headers=headers)
    return r



In [9]:
# export


def create_contacts(contacts=None, hapikey=None, check_exists=True):
    """
    Create contacts from a list of contacts.
    """
    if hapikey is None:
        hapikey = os.environ.get('HUBSPOT_API_KEY','demo')
    url = "https://api.hubapi.com/contacts/v1/contact/batch"
    querystring = {"hapikey": hapikey}
    headers = {"Content-Type": "application/json"}
    payload = []
    # loop over contacts and build payload to send
    for contact in contacts:
        email = contact["email"]
        payload.append({"email": email, "properties": [{"property": k,"value": contact[k]} for k in contact]})
    payload = json.dumps(payload)
    r = requests.request("POST", url, data=payload, headers=headers, params=querystring)
    if r.status_code == 202:
        return True
    else:
        raise ValueError(f"batch contact creation failed status_code={r.status_code} reason={r.reason}")
        return False



In [10]:
# export


def update_contact(email, properties, hapikey=None):
    """
    Update a contact.
    """
    if hapikey is None:
        hapikey = os.environ.get('HUBSPOT_API_KEY','demo')
    if contact_exists(email):
        contact = get_contact(email)
        vid = contact['id']
        endpoint = 'https://api.hubapi.com/contacts/v1/contact/'
        url = f'{endpoint}/vid/{vid}/profile?hapikey={hapikey}'
        headers = {
            "Content-Type": "application/json"
        }
        data = json.dumps({"properties": [{"property": k,"value": properties[k]} for k in properties]})
        r = requests.post(url=url, data=data, headers=headers)
        return r
    else:
        return None



In [11]:
# export


def create_or_update_contact(email, properties, hapikey=None):
    """
    Create or update a contact.
    """
    if hapikey is None:
        hapikey = os.environ.get('HUBSPOT_API_KEY','demo')
    endpoint = 'https://api.hubapi.com/contacts/v1/contact/'
    url = f'{endpoint}/createOrUpdate/email/{email}?hapikey={hapikey}'
    headers = {
        "Content-Type": "application/json"
    }
    data = json.dumps({"properties": [{"property": k,"value": properties[k]} for k in properties]})
    r = requests.post(url=url, data=data, headers=headers)
    return r



In [12]:
# hide
# tests

test_email = 'fake_user@example.com'

# first delete test contact if exists
contact = get_contact(test_email)
if contact:
    delete_contact(vid=contact['id'])

time.sleep(5)

# create an example minimal contact
contact_data = {
    "email": test_email
}

# create contact
response_create = create_contact(contact_data)
assert 'vid' in response_create.json()

time.sleep(5)

# check it exists
assert contact_exists(test_email) == True

time.sleep(2)

# get contact
contact = get_contact(test_email)
assert contact['properties']['email'] == test_email

time.sleep(2)

# update contact
response_update = update_contact(test_email, {"firstname": "fake", "lastname": "user"})
assert response_update.status_code == 204

time.sleep(5)

contact = get_contact(test_email)
assert contact['properties']['firstname'] == 'fake'
assert contact['properties']['lastname'] == 'user'

time.sleep(2)

# test create_or_update existing
response = create_or_update_contact(test_email, {"firstname": "fake_exists"}, hapikey)
assert response.json()['isNew'] == False

time.sleep(2)

# delete contact
response_delete = delete_contact(vid=contact['id'])
assert response_delete.status_code == 200

time.sleep(2)

# test create_or_update new
response = create_or_update_contact(test_email, {"firstname": "fake_new"}, hapikey)
assert response.json()['isNew'] == True
time.sleep(2)
contact = get_contact(test_email)
response_delete = delete_contact(vid=contact['id'])
assert response_delete.status_code == 200

## create batch of contacts
#test_contacts = [
#    dict(email='user1@example.com'),
#    dict(email='user2@example.com'),
#]
#assert create_contacts(test_contacts) == True
#time.sleep(5)
## now clean up and delete them
#for test_contact in test_contacts:
#    time.sleep(5)
#    test_email = test_contact['email']
#    test_contact_vid = get_contact(test_email)['id']
#    delete_contact(vid=test_contact_vid)